https://github.com/philipperemy/timit

https://www.kaggle.com/datasets/mfekadu/darpa-timit-acousticphonetic-continuous-speech?resource=download

https://www.kaggle.com/code/julwan/phoneme-recognition-with-wav2vec2

wav2vec seems like a common transformer model for speech recognition. 

630 speakers, each read 10 sentences—2 dialect "shibboleth" sentences (SA), 5 phonetically compact sentences (SX), and 3 phonetically diverse sentences (SI).

- SA: expose the dialectal variants of the speakers.
- SX: provide a good coverage of pairs of phones, with extra occurrences of phonetic contexts thought to be either difficult or of particular interest.
- SI: add diversity in sentence types and phonetic contexts.



| Sentence Type | #Sentences | #Speakers | Total | #Sentences/Speaker |
|---------------|------------|-----------|-------|--------------------|
| Dialect (SA)  |      2     |    630    | 1260  |         2          |
| Compact (SX)  |     450    |     7     | 3150  |         5          |
| Diverse (SI)  |    1890    |     1     | 1890  |         3          |
|---------------|------------|-----------|-------|--------------------|
| Total         |    2342    |           | 6300  |        10          |


| Dialect Region (dr) | #Male   | #Female  | Total   |
|----------------------|---------|----------|---------|
|         1            | 31 (63%)| 18 (27%) | 49 (8%) |
|         2            | 71 (70%)| 31 (30%) |102(16%)|
|         3            | 79 (67%)| 23 (23%) |102(16%)|
|         4            | 69 (69%)| 31 (31%) |100(16%)|
|         5            | 62 (63%)| 36 (37%) | 98(16%)|
|         6            | 30 (65%)| 16 (35%) | 46 (7%)|
|         7            | 74 (74%)| 26 (26%) |100(16%)|
|         8            | 22 (67%)| 11 (33%) | 33 (5%)|
|----------------------|---------|----------|---------|
|        Total         | 438(70%)|192(30%)  |630(100%)|

The dialect regions are:
   dr1: New England
   dr2: Northern
   dr3: North Midland
   dr4: South Midland
   dr5: Southern
   dr6: New York City
   dr7: Western
   dr8: Army Brat (moved around)


For each utterance, the following information is available:
- .wav file: the speech waveform.
- .phn file: Time-aligned phonetic transcription.
- .wrd file: Time-aligned word transcription.
- .txt file: Associated orthographic transcription of the words the
            person said.

In [150]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
import glob
import librosa
import pandas as pd

# custum modules
from data_utils import *

ideal data structure:
``` python
data = {
    "document1": {
        "word": pd.DataFrame, 
        "phoneme": pd.DataFrame}
    ,
    "document2": {
        "word": pd.DataFrame,
        "phoneme": pd.DataFrame
    }
}
```

## Load phoneme, word, and transcript data

In [193]:
# sx61
glob.glob('timit/data/TRAIN/DR4/MGAG0/SX61.*')

['timit/data/TRAIN/DR4/MGAG0/SX61.TXT',
 'timit/data/TRAIN/DR4/MGAG0/SX61.WAV',
 'timit/data/TRAIN/DR4/MGAG0/SX61.PHN',
 'timit/data/TRAIN/DR4/MGAG0/SX61.WRD',
 'timit/data/TRAIN/DR4/MGAG0/SX61.WAV.wav']

In [85]:
# phoneme
df_phoneme = load_data(glob.glob('timit/data/TRAIN/DR4/MGAG0/SX61.PHN')[0], "phoneme")
df_phoneme.tail()

,start_sample,end_sample,phoneme,diff_sample
32,36940,39000,#b,2060
33,39000,40200,#b,1200
34,40200,40960,#b,760
35,40960,41720,#b,760
36,41720,43680,h#,1960


In [195]:
# word
df_word = load_data(glob.glob('timit/data/TRAIN/DR4/MGAG0/SX61.WRD')[0], "word")
df_word.tail()

,start_sample,end_sample,word,diff_sample
4,20305,26520,fail,6215
5,26520,28393,as,1873
6,28393,29033,a,640
7,29033,36280,romantic,7247
8,36280,41720,gift,5440


In [196]:
df_transcript = load_transcript('timit/data/TRAIN/DR4/MGAG0/SX61.TXT')
df_transcript.tail()

,start_sample,end_sample,transcript,diff_sample
0,0,43725,Chocolate and roses never fail as a romantic g...,43725


## MFCC
Note that we can always use smaller `win_length` and `hop_length` for more fine-grained alignment.

if we choose 25ms window and 5ms hop, we can have $25 * 10^-3 * 16000 = 400$ `win_length` and $5 * 10^-3 * 16000 = 80$ `hop_length`.

https://librosa.org/doc/main/generated/librosa.feature.mfcc.html#librosa.feature.mfcc

In [131]:
df_mfcc = process_audio_file('timit/data/TRAIN/DR4/MGAG0/SX61.WAV',
                             win_length=400, 
                             hop_length=80)
df_mfcc.tail()

,start_sample,end_sample,mfcc
541,43280,43680,"[-827.1031, 32.212986, 10.357775, 4.273916, -7.0693197, 0.10315089, -2.513186, -4.818401, -1.4071487, 8.893745, 11.381563, 7.5945396, 1.589353, 5.308113, 5.909975, -4.874711, -6.9147296, -0.20854214, 2.7190595, -4.6186132]"
542,43360,43760,"[-833.8036, 18.833408, 6.527279, 11.398876, 0.103251174, 0.8042935, 3.4333205, -4.118975, -3.5335937, 4.9041696, 8.634795, 5.300904, 7.984957, 11.643053, 4.904706, -4.5845375, 1.6711016, 6.4817266, 0.8232577, -5.371475]"
543,43440,43840,"[-845.2821, 3.8058267, 1.8199315, 11.270794, 5.8475585, -1.3674686, 2.779904, -9.955495, -3.1452274, 1.7140839, 5.052147, -0.19389728, 5.7216744, 8.643326, 0.7713368, 3.1126137, 10.645617, 11.829777, 8.475585, 5.1200647]"
544,43520,43920,"[-847.30914, 4.7726836, 6.2668734, 6.074581, 8.445786, -0.121844046, 0.40512764, -4.014574, 6.1389318, 1.0904772, -0.5691086, -4.44184, 5.037013, 10.637453, 5.1548405, 11.605476, 4.642511, 0.25984132, 11.158846, 11.875011]"
545,43600,44000,"[-847.56964, 9.320388, 15.307094, 7.9442987, 10.220013, 7.1344895, 3.296341, 6.3600864, 11.376036, 8.035162, 7.2890534, -1.6766523, 7.5888567, 8.748432, 10.811941, 20.659288, 4.826117, 0.39584345, 2.2651634, 3.354187]"


In [63]:
df_mfcc.shape

(546, 3)

In [204]:
400/16000

0.025

In [205]:
80/16000

0.005

### draft/explanation

y: total number of samples

sr: number of samples per second

In [ ]:
y, sr = librosa.load('timit/data/TRAIN/DR4/MGAG0/SX61.WAV', sr=16000)
y.shape, sr

((43725,), 16000)

In [200]:
y, sr = librosa.load('timit/data/TRAIN/DR4/MGAG0/SX61.WAV')
y.shape, sr

((60259,), 22050)

In [201]:
# duration
60259/22050

2.732834467120181

we know y=43725

In [198]:
load_transcript('timit/data/TRAIN/DR4/MGAG0/SX61.TXT')

,start_sample,end_sample,transcript,diff_sample
0,0,43725,Chocolate and roses never fail as a romantic g...,43725


In [197]:
# try another 
y, sr = librosa.load('timit/data/TRAIN/DR4/MGAG0/SA1.WAV', sr=16000)
y.shape, sr

((45159,), 16000)

In [ ]:
load_transcript('timit/data/TRAIN/DR4/MGAG0/SA1.TXT')

,start_time,end_time,transcript
0,0,45159,She had your dark suit in greasy wash water al...


In [ ]:
# Number of samples=Duration×Sampling frequency (Sr in Hz)
# Hz represents the number of samples per second.
duration = len(y) / sr
duration

2.732834467120181

In [ ]:
43725/duration  
# if an audio file is 2.732834467120181 seconds long and contains 43725 samples, it means that the audio file has a sampling rate of approximately 16000 Hz

15999.871388506283

In [ ]:
n_mfcc = 20  # Number of MFCC coefficients
hop_length = 512  # Number of samples between consecutive frames (frame step)
win_length = 1024  # Length of the analysis window in samples (frame size)

In [ ]:
1024/16000  # 64 ms

0.064

In [ ]:
# Extract MFCC features
mfccs = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, win_length=win_length, n_mfcc=n_mfcc)

In [ ]:
mfccs.shape
# The first dimension (20) represents the number of MFCC coefficients extracted. This is a common default value, as 20 coefficients are often sufficient to capture relevant information about the spectral characteristics of the audio signal.
# The second dimension (86) represents the number of frames extracted from the audio signal. The number of frames is determined by the duration of the audio signal and the frame length and hop length used in the feature extraction process.

(20, 86)

In [ ]:
mfcc = load_mfcc('timit/data/TRAIN/DR4/MGAG0/SX61.WAV')
mfcc.shape

(20, 86)

In [ ]:
df_mfcc = construct_mfcc_df(mfcc)
df_mfcc.tail()

,start_sample,end_sample,mfcc
81,41472,42496,"[-494.7277, 7.1650486, -1.718417, 17.082987, -..."
82,41984,43008,"[-627.15405, 57.744026, -20.17612, 10.34127, -..."
83,42496,43520,"[-698.2599, 51.94442, -1.7764928, 8.506399, 11..."
84,43008,44032,"[-753.46246, 40.802723, 8.634056, 1.6109326, 1..."
85,43520,44544,"[-795.0889, 12.04735, 9.87564, 9.984465, 5.128..."


## Align df_mfcc and df_phoneme 

epi, pau, h# are silence.

In [101]:
target_phoneme = ['epi', 'pau', 'h#', 'n', 'eh', 'v', 'axr']

# substitute every other phoneme with "#b"
df_phoneme['phoneme'] = df_phoneme['phoneme'].apply(lambda x: "#b" if x not in target_phoneme else x)
df_phoneme['phoneme'] = df_phoneme['phoneme'].apply(lambda x: "h#" if x in ['epi', 'pau'] else x)
df_phoneme.head()


,start_sample,end_sample,phoneme,diff_sample
0,0,2370,h#,2370
1,2370,3442,#b,1072
2,3442,5351,#b,1909
3,5351,5973,#b,622
4,5973,6863,#b,890


In [189]:
df_phoneme.loc[13:18]

,start_sample,end_sample,phoneme,diff_sample
13,15882,16790,#b,908
14,16790,17642,n,852
15,17642,18673,eh,1031
16,18673,19342,v,669
17,19342,20305,axr,963
18,20305,22950,#b,2645


In [102]:
target_range = df_mfcc[(df_mfcc['end_sample'] >= df_phoneme.loc[14, 'start_sample']) & (df_mfcc['start_sample'] <= df_phoneme.loc[18, 'end_sample'])]
target_range

,start_sample,end_sample,mfcc
205,16400,16800,"[-489.03186, -12.311717, 58.93512, 42.58722, -..."
206,16480,16880,"[-504.2323, -29.32409, 67.523476, 37.44905, -1..."
207,16560,16960,"[-521.99133, -28.142538, 59.3443, 28.163754, -..."
208,16640,17040,"[-531.2007, -17.439379, 42.591145, 7.9479475, ..."
209,16720,17120,"[-558.5072, -11.389967, 43.460793, -2.4091964,..."
210,16800,17200,"[-623.5903, -1.6005177, 46.49632, -1.0682647, ..."
211,16880,17280,"[-666.8244, 19.64281, 33.50325, 21.436499, -4...."
212,16960,17360,"[-649.2126, 45.17038, 34.706863, 23.450588, -8..."
213,17040,17440,"[-639.8352, 71.87477, 52.9447, 32.282722, -11...."
214,17120,17520,"[-636.0895, 101.487274, 63.298653, 34.73152, -..."


Because a frame (400 samples) can never contain more than 2 phonemes, we can use the following data structure to store the phoneme information.

```
h# (silence)
#b (other phonemes, noise)
#bn (bi-phonemes)
n
neh (bi-phonemes)
eh
ehv
v
axr
vaxr (bi-phonemes)
axr#b (bi-phonemes)
```

11 dimensional feature vector for each frame.

In [95]:

df_phoneme

,start_sample,end_sample,phoneme,diff_sample
0,0,2370,h#,2370
1,2370,3442,#b,1072
2,3442,5351,#b,1909
3,5351,5973,#b,622
4,5973,6863,#b,890
5,6863,7118,#b,255
6,7118,7689,#b,571
7,7689,8871,#b,1182
8,8871,10485,#b,1614
9,10485,11947,#b,1462


In [132]:
df_mfcc

,start_sample,end_sample,mfcc
0,0,400,"[-855.473, 12.425409, 30.045187, 20.633747, 14.063944, 10.741582, 14.659479, 0.33916304, 10.186453, -4.282422, 3.6793895, 5.4730883, 0.011911005, -2.0800016, 1.8878455, 9.833944, -1.6392567, 0.17326328, -4.0900116, -13.733365]"
1,80,480,"[-845.2309, 12.037045, 26.38384, 24.935616, 15.311946, 8.823542, 11.616955, -0.59074366, 8.866205, -5.653927, -0.12149161, 7.2000093, 3.0656204, -6.310438, 3.4328513, 11.944434, -0.7053368, 2.8076825, 1.2786763, -12.331352]"
2,160,560,"[-847.46045, 10.093206, 16.57772, 15.377459, 11.035589, 4.864591, 12.584749, 7.7981644, 5.2682066, -5.470523, 4.9896226, 6.9582267, 8.020832, 1.0450811, 15.5572195, 12.722178, 3.7916129, 11.090977, 4.880035, -10.939484]"
3,240,640,"[-836.5919, 18.25502, 14.159644, 5.1573267, 8.6434355, 9.274525, 7.599383, 1.9540653, -5.805154, -7.7468767, 5.4599056, 0.10474473, 2.694273, 4.14482, 16.103273, 12.51374, 3.0500712, 7.4298677, 7.422206, -0.89379996]"
4,320,720,"[-839.7757, 22.763817, 20.63623, 3.520937, 4.231862, 10.286663, 5.1169825, 0.4790709, -7.2799764, -0.11986989, 5.384306, -3.0221934, 6.6694717, 6.068678, 3.357371, 4.9890075, 1.9164655, 2.7152288, 6.049534, 11.539325]"
5,400,800,"[-846.547, 11.539566, 19.615639, 11.0121765, 9.643043, 9.836796, 9.7821, 10.306385, 5.877302, 7.2996807, 8.21134, 0.7093023, 9.052889, 6.797043, 3.2460134, -4.873557, -0.7694579, 1.331023, -0.31671515, 10.427031]"
6,480,880,"[-846.9009, 8.728346, 14.260023, 12.773836, 8.044616, 1.533608, 6.79418, 9.017656, 6.1151905, 4.93141, 10.749659, 5.648065, 10.805996, 14.1089735, 12.586565, -3.3409452, 2.72835, 2.7290456, 1.5485461, 5.642045]"
7,560,960,"[-846.14813, 10.737683, 15.2987385, 17.72052, 7.688822, -4.824424, 8.443262, 3.3873665, 2.9744215, 3.6022882, 9.802633, 5.858921, 5.7050257, 12.52363, 6.8677998, 1.2055811, 1.327734, -0.68597955, 3.738205, 0.6022278]"
8,640,1040,"[-844.6941, 7.2958155, 10.302032, 12.729725, 4.4596925, -0.46815073, 16.576735, 6.1090393, 5.3081775, 6.492007, 5.1971445, 2.2685091, -1.263668, 11.272411, 11.240147, 11.926428, 2.32403, 4.907749, 9.992794, 2.46386]"
9,720,1120,"[-847.8949, 4.3166027, 11.775213, 8.45533, 4.607132, 3.3250394, 7.2067966, -1.014581, 6.0214534, 12.770563, 8.177901, 9.597971, 1.8201454, 4.537004, 12.738824, 15.891458, 1.9852748, 1.9969845, -0.04141943, 2.0417502]"


Example: 'n, eh'
Option 1: 50% probability to intersection 'neh'. The remaining is 'n' and 'eh'. 
Option 2: 

In [136]:
# Adjust display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

new_df_mfcc = label_df_mfcc(df_mfcc, df_phoneme) 
new_df_mfcc[(df_mfcc['start_sample']>=16000) & (df_mfcc['end_sample']<=21000)] 

,start_sample,end_sample,mfcc,phoneme
200,16000,16400,"[-484.46353, 62.9625, 43.536667, 36.161552, -33.775795, 7.7863464, -7.81036, -32.905396, 0.38289544, 6.5445538, -3.362215, -19.924276, -11.869465, 11.452057, 8.279446, -4.9956255, 0.43326283, 6.2694674, -7.0054655, 1.9755795]",#b
201,16080,16480,"[-493.01233, 40.778465, 48.444675, 40.07834, -23.769608, 17.849758, -14.231265, -26.100803, 4.215081, 9.904649, -1.5833447, -25.324093, -15.639889, 9.591164, 6.5180416, -3.9707384, 3.0204115, 7.1835256, -6.532056, 6.8589716]",#b
202,16160,16560,"[-496.74826, 30.404907, 50.27643, 51.164154, -18.135803, 14.162009, -17.96069, -22.23743, 3.14307, 4.2015414, -6.2886233, -20.614836, -12.755966, 6.4162655, 2.4632564, -4.287204, 13.980566, 10.460466, -11.342501, 1.995001]",#b
203,16240,16640,"[-486.746, 15.234005, 45.627895, 56.397243, -22.324577, 1.3790563, -14.829262, -9.336773, 0.7815729, 6.828738, -5.83054, -16.375328, -7.6271615, 12.749347, 2.4277577, -11.2310505, 15.654282, 6.4201384, -8.722306, 1.0047979]",#b
204,16320,16720,"[-480.75903, 9.849446, 51.09784, 55.48481, -24.305824, 3.967523, -16.57162, -12.115267, -6.520217, 7.7633343, 5.4241953, -15.951579, -15.844574, 11.521286, 4.755411, -9.295454, 10.152107, -1.1387705, -3.4536805, 5.669768]",#b
205,16400,16800,"[-489.03186, -12.311717, 58.93512, 42.58722, -19.939394, 16.146267, -22.301338, -21.727514, 0.28492898, 6.2198315, 4.7558155, -15.748632, -9.075333, 11.358965, 5.0113773, -0.22935735, 13.739038, -1.828378, -5.681775, -1.4469748]","#b, n"
206,16480,16880,"[-504.2323, -29.32409, 67.523476, 37.44905, -13.633593, 23.534386, -28.00717, -22.00079, 14.396887, 8.668379, 2.1295342, -6.4604816, -1.8816123, 1.2565889, -2.6831994, -2.1007128, 10.023035, 2.7754722, 0.71725273, -5.889139]","#b, n"
207,16560,16960,"[-521.99133, -28.142538, 59.3443, 28.163754, -9.627581, 28.722317, -24.941507, -10.856624, 16.647911, -1.9732568, -4.1634145, -7.0630097, -4.5952353, -4.6961694, -6.205681, -11.348597, 8.628605, 0.5363274, -2.7605577, 0.04995489]","#b, n"
208,16640,17040,"[-531.2007, -17.439379, 42.591145, 7.9479475, -19.197601, 18.446426, -31.931095, -14.686649, 2.0308957, -7.3065987, 11.542793, -4.302381, -0.12107611, 13.267833, 5.3845015, -14.3902, 6.634276, -12.831449, -7.687578, 7.503215]","#b, n"
209,16720,17120,"[-558.5072, -11.389967, 43.460793, -2.4091964, -20.158928, 11.41258, -26.897453, -16.792953, -16.090136, -4.617997, 32.536724, 9.14871, 8.515188, 22.414402, 3.7902784, -8.4484825, 9.170767, -9.301361, 0.8911765, 8.615245]","#b, n"


In [141]:
new_df_mfcc2 = vectorize_label_df_mfcc(new_df_mfcc, df_phoneme)
new_df_mfcc2[(new_df_mfcc2['start_sample']>=16000) & (new_df_mfcc2['end_sample']<=21000)] 

,start_sample,end_sample,mfcc,phoneme,phoneme_weights,distributed_weights,label
200,16000,16400,"[-484.46353, 62.9625, 43.536667, 36.161552, -33.775795, 7.7863464, -7.81036, -32.905396, 0.38289544, 6.5445538, -3.362215, -19.924276, -11.869465, 11.452057, 8.279446, -4.9956255, 0.43326283, 6.2694674, -7.0054655, 1.9755795]",#b,{'#b': 1.0},{'#b': 1.0},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
201,16080,16480,"[-493.01233, 40.778465, 48.444675, 40.07834, -23.769608, 17.849758, -14.231265, -26.100803, 4.215081, 9.904649, -1.5833447, -25.324093, -15.639889, 9.591164, 6.5180416, -3.9707384, 3.0204115, 7.1835256, -6.532056, 6.8589716]",#b,{'#b': 1.0},{'#b': 1.0},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
202,16160,16560,"[-496.74826, 30.404907, 50.27643, 51.164154, -18.135803, 14.162009, -17.96069, -22.23743, 3.14307, 4.2015414, -6.2886233, -20.614836, -12.755966, 6.4162655, 2.4632564, -4.287204, 13.980566, 10.460466, -11.342501, 1.995001]",#b,{'#b': 1.0},{'#b': 1.0},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
203,16240,16640,"[-486.746, 15.234005, 45.627895, 56.397243, -22.324577, 1.3790563, -14.829262, -9.336773, 0.7815729, 6.828738, -5.83054, -16.375328, -7.6271615, 12.749347, 2.4277577, -11.2310505, 15.654282, 6.4201384, -8.722306, 1.0047979]",#b,{'#b': 1.0},{'#b': 1.0},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
204,16320,16720,"[-480.75903, 9.849446, 51.09784, 55.48481, -24.305824, 3.967523, -16.57162, -12.115267, -6.520217, 7.7633343, 5.4241953, -15.951579, -15.844574, 11.521286, 4.755411, -9.295454, 10.152107, -1.1387705, -3.4536805, 5.669768]",#b,{'#b': 1.0},{'#b': 1.0},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
205,16400,16800,"[-489.03186, -12.311717, 58.93512, 42.58722, -19.939394, 16.146267, -22.301338, -21.727514, 0.28492898, 6.2198315, 4.7558155, -15.748632, -9.075333, 11.358965, 5.0113773, -0.22935735, 13.739038, -1.828378, -5.681775, -1.4469748]","#b, n","{'#b': 0.4875, 'n': 0.0125, '#bn': 0.5}","{'#b': 0.4875, 'n': 0.0125, '#bn': 0.5}","[0.5, 0.0125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4875, 0.0]"
206,16480,16880,"[-504.2323, -29.32409, 67.523476, 37.44905, -13.633593, 23.534386, -28.00717, -22.00079, 14.396887, 8.668379, 2.1295342, -6.4604816, -1.8816123, 1.2565889, -2.6831994, -2.1007128, 10.023035, 2.7754722, 0.71725273, -5.889139]","#b, n","{'#b': 0.3875, 'n': 0.1125, '#bn': 0.5}","{'#b': 0.3875, 'n': 0.1125, '#bn': 0.5}","[0.5, 0.1125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3875, 0.0]"
207,16560,16960,"[-521.99133, -28.142538, 59.3443, 28.163754, -9.627581, 28.722317, -24.941507, -10.856624, 16.647911, -1.9732568, -4.1634145, -7.0630097, -4.5952353, -4.6961694, -6.205681, -11.348597, 8.628605, 0.5363274, -2.7605577, 0.04995489]","#b, n","{'#b': 0.2875, 'n': 0.2125, '#bn': 0.5}","{'#b': 0.2875, 'n': 0.2125, '#bn': 0.5}","[0.5, 0.2125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2875, 0.0]"
208,16640,17040,"[-531.2007, -17.439379, 42.591145, 7.9479475, -19.197601, 18.446426, -31.931095, -14.686649, 2.0308957, -7.3065987, 11.542793, -4.302381, -0.12107611, 13.267833, 5.3845015, -14.3902, 6.634276, -12.831449, -7.687578, 7.503215]","#b, n","{'#b': 0.1875, 'n': 0.3125, '#bn': 0.5}","{'#b': 0.1875, 'n': 0.3125, '#bn': 0.5}","[0.5, 0.3125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1875, 0.0]"
209,16720,17120,"[-558.5072, -11.389967, 43.460793, -2.4091964, -20.158928, 11.41258, -26.897453, -16.792953, -16.090136, -4.617997, 32.536724, 9.14871, 8.515188, 22.414402, 3.7902784, -8.4484825, 9.170767, -9.301361, 0.8911765, 8.615245]","#b, n","{'#b': 0.0875, 'n': 0.4125, '#bn': 0.5}","{'#b': 0.0875, 'n': 0.4125, '#bn': 0.5}","[0.5, 0.4125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0875, 0.0]"


In [127]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

In [175]:
# locate the range of phoneme
start_sample = row['start_sample']
end_sample = row['end_sample']

phoneme_vector = [0] * 10

phonme_subset = df_phonme[(df_phonme['start_sample'] >= start_sample) & (df_phonme['end_sample'] <= end_sample)]

for _, phonme_row in phonme_subset.iterrows():
    phoneme = phonme_row['phoneme']
    if phoneme in phonemes:
        phoneme_index = phonemes.index(phoneme)
        phoneme_vector[phoneme_index] = 1

phoneme_labels.append(phoneme_vector)

phoneme_labels

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]